In [1]:
%load_ext autoreload
%autoreload 2


In [3]:
import os
print(os.getcwd())

/opt/spark/work-dir/notebooks


In [4]:
import sys
from spark.spark_config import create_spark_session
spark = create_spark_session()

ModuleNotFoundError: No module named 'py4j'

In [5]:
spark

In [6]:
from data_loader import DataLoader
from preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from feature_extractor import FeatureExtractor


In [7]:
data_loader = DataLoader(spark)
df = data_loader.load_sample()

In [8]:
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])


In [9]:
df = Preprocessor.run(df)


In [10]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [11]:
extractor = FeatureExtractor(vocabSize=10000, minDF=5.0)
extractor.fit(df_tokenized)
df_final = extractor.transform(df_tokenized)
df_tokenized.unpersist()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [12]:
# from pathlib import Path
# base_dir = Path.cwd().parent
# test_file_path = base_dir / "models" / "test" / "tf-idf"
# # extractor.save(str(test_file_path))
# extractor.load(str(test_file_path))

In [13]:
df_final.show(3)

[Stage 7:>                                                          (0 + 1) / 1]

+--------------------+---+--------------------+----+-----+--------------------+--------------------+-----+-------------+-----------+--------+--------------------+--------------------+--------------------+
|               title|tag|              artist|year|views|            features|              lyrics|   id|language_cld3|language_ft|language|      lyrics_cleaned|    words_lemmatized|         tfidf_array|
+--------------------+---+--------------------+----+-----+--------------------+--------------------+-----+-------------+-----------+--------+--------------------+--------------------+--------------------+
|CJ KaZet - My wor...|rap|Polskie tumaczeni...|2011|   42|{"Polskie tłumacz...|(intro)\nSłuchaj,...|53105|           pl|         pl|      pl|Słuchaj, czy zast...|[słuchaj, zastana...|[0.0, 0.0, 0.0, 0...|
|               Gniew|rap|              Pi Dwa|2008| 4883|        {"Pięć Dwa"}|[Intro]\n- Czy Ty...|58000|           pl|         pl|      pl|- Czy Ty nie masz...|[mieć, żaden, ucz.